In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
input_train=[]
y=[]
input_test=[]
y_test=[]
with open("../input/cloning.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t")
    next(rd)
    for raw in rd:
        if raw[-1]=='train' or raw[-1]=='val':
            input_train.append(raw[4])
            y.append(raw[5])
        else:
            input_test.append(raw[4])
            y_test.append(raw[5])
        
print(len(input_train))

In [ ]:
def one_hot_y(y):
    out_train=[]
    for i in y:
        if i=='NoArgument':
            out_train.append([1,0,0])
        elif i=='Argument_for':
            out_train.append([0,1,0])
        elif i=='Argument_against':
            out_train.append([0,0,1])
    return out_train

In [ ]:
out_train=one_hot_y(y)
out_test=one_hot_y(y_test)

In [ ]:
input_train=input_train
input_test=input_test
out_train=np.asarray(out_train)
out_test=np.asarray(out_test)
print(len(input_train))
print (out_train.shape)
print(len(input_test))
print(out_test.shape)

In [ ]:
import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
#pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
#train_data = x
#pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv')
#test_data = r
#train_data["comment_text"].fillna("fillna")
#test_data["comment_text"].fillna("fillna")
#input_train = x
#train_data["comment_text"].str.lower()
#output_train = r
#train_data[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

#input_test = test_data["comment_text"].str.lower()

max_features=1000
maxlen=150
embed_size=300
tok=text.Tokenizer(num_words=max_features,lower=True)
tok.fit_on_texts(list(input_train)+list(input_test))
input_train=tok.texts_to_sequences(input_train)
input_test=tok.texts_to_sequences(input_test)
X_train = pad_sequences(input_train, maxlen=maxlen)
X_test=pad_sequences(input_test, maxlen=maxlen)
#input_train[:]
print(X_train.shape)
print(X_train[0].shape)

In [ ]:

inp = Input(shape=(maxlen, ))
print(inp.shape)
x = Embedding(max_features, embed_size)(inp)#maxlen=200 as defined earlier
print(x.shape)
x = Bidirectional(LSTM(96, return_sequences=True,dropout=0.15,recurrent_dropout=0.15))(x)
#x = Bidirectional(LSTM(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
print(x.shape)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])
preds = Dense(3, activation="sigmoid")(x)
print(preds.shape)
model = Model(inp, preds)
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])

batch_size = 128
epochs = 7


In [ ]:
#filepath="../input/weights_base.best.hdf5"
#checkpoint = ModelCheckpoint(weights_base.best.hdf5, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#early = EarlyStopping(monitor="val_acc", mode="max", patience=5)


In [ ]:
model.fit(X_train, out_train, batch_size=batch_size,validation_data=(X_test, out_test), epochs=epochs,verbose=1)
model.summary()
model.save('my_model.h5')
